In [2]:
from hashlib import blake2b 
class MapResult():
    
    import pickle
    import math
    statistika = None
    words1= None
    words2 = None
    emb1 = ""
    emb2 = ""
    vzdalenostniMapa = None
    embeddingy = None
    dtype = [("avg",float),("index",float),("nRecords",float)]
    words = None 
    
    def __init__(self,emb1 = "", emb2 = "", words1 = None, words2=None,vzdalenostniMapa=None,text = ""):
        self.emb1 = emb1
        self.emb2 = emb2
        self.words1 = words1
        self.words2 = words2
        self.vzdalenostniMapa = vzdalenostniMapa
        if(text != ""):
            self.words = text
            embeddingy = Embeddings_ready(text=text)
            #self.emb1 = embeddingy
            #self.emb1 = #TODO - Hledání + načítání z disku podle názvu / hashe / db?... prostě něco
            self.vzdalenostniMapa = embeddingy.generateMap()
    
    def toString(self):
        return("words1: " + str(self.words1 or "None") + "\n" +
              "words2: " + str(self.words2 or "None") + "\n" +
               "emb1: " + str(self.emb1) + "\n"+
               "emb2: " + str(self.emb2) + "\n"+
               "statistika: " + str(self.statistika or "None") + "\n"+
               "vzdalenostniMapa: " + str(self.vzdalenostniMapa or "None") + "\n"              )
        
  
    
 
    def getHash(self, words:str = None):
        hasher = blake2b()
        if (self != None):
            if(self.words == None):
                words = self.words1+self.words2
            else: 
                words=self.words
            
        hasher.update(bytes(words,"UTF-8"))
        return hasher.hexdigest()
    
    def getEmbeddings(self,number:int = 0):
        if(number == 0):
            if(self.emb1==""):
                return None #Asi by měl Raisnout Exception
            return pickle.load(open(self.emb1,"rb"))
                
        elif(number == 1):
            if(self.emb2==""):
                if(self.emb1==""):
                    return None #Asi by měl Raisnout Exception
                return pickle.load(open(self.emb1,"rb"))
            return pickle.load(open(self.emb2,"rb")) 
            
        return None #DEFAULT 
    
    def getVzdalenostniMapa(self):
        if(self.vzdalenostniMapa == None):
            self.generateMap()
        return self.vzdalenostniMapa
    
    def generateMap(self,matrix=None,firstSEC:int = 0, secondSEC:int = 1):
        if(matrix == None):
            
            matrix = self.getEmbeddings().getMapReadyEmbeddings(anotherClass=self.getEmbeddings(number=1),
                                                                firstSEC=firstSEC,
                                                                secondSEC=secondSEC)
        retVal = []
        for i in range(np.size(matrix,0)):
            retVal.append([])
            for j in range(np.size(matrix,1)):
                retVal[i].append(modelDist(matrix[i][j])[1][0][0].numpy()[0])
        #print(modelDist(matrix[i][j])[1][0][0].numpy()[0])
        self.vzdalenostniMapa= retVal
    
    
    def Statistics(self, mapaData=None):
        #TODO: Další statistické funkce, možná úprava ztrátové funkce, či něco dalšího
        if(mapaData ==None):
            mapaData = self.getVzdalenostniMapa()
        nRecords = np.size(mapaData,0)-1
        rangeMax = round(nRecords/2)
        #best_avg = 0
        #pos_avg = 0
        mapaData2 = np.array(mapaData)
        all_Avgs = []
        for i in range(-rangeMax,rangeMax): 
            #Validation_bias - "ztrátová funkce" pro průměry mimo očekávanou osu
            #validation_bias = 1-abs((i+30)/nRecords)
            all_Avgs.append((np.average(mapaData2.diagonal(i)),i,rangeMax-i))
        return all_Avgs
    
    def prumery(self,statistika):
        np.flip(np.sort(np.array(statistika,dtype=self.dtype),order="avg"))
    
    def pickleThis(self,path:str = "./Pickles"):
        hash = self.getHash()
        with open(path+"/"+hash+".pickle","wb") as p:
            pickle.dump(self,p)
            p.close()
        
    

In [3]:
def heatmapa(vstup1,vstup2,delimiter):
    
    
    splittedString1 = textSplitter(vstup1,delimiter)
    splittedString2 = textSplitter(vstup2,delimiter)
    
    
    delka1 = len(splittedString1)                
    delka2 = len(splittedString2)

    
    
    mapa = np.zeros((delka1,delka2))
    for i in range(delka1-1):
    #X
        for j in range(delka2-1):
            #Y
            #print(splittedString[i] + " " + splittedString[i+1]     )
            batch = []
            batch.append(((splittedString1[i],splittedString1[i+1],splittedString2[j],splittedString2[j+1]),0))
            vectorized = vectorize_batch(batch,mask=False)        
            out = model.predict(vectorized,batch_size = 32)
            mapa[i,j+1] = out["out"][0][3]
            mapa[i+1,j] = out["out"][0][2]
    return mapa

def textSplitter(vstup, condition):
    #Musí být ve stejné velikosti jako výukové okno - 32
    #NEW:
    vstup += ' '*32
    vstup.replace('\n',"")
    if(condition.isnumeric()):
        cislo = int(condition)
        if(len(vstup)<32):
            #print(len(vstup))
            return [vstup]        
        return [vstup[i:i+32] for i in range(0,len(vstup)-32,cislo)]
        
        
    else:
        return vstup.split(condition)

In [4]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib as mpl
def Mapa(vstup,exportName:str = "",title:str = ""):
    import matplotlib.pyplot as plt
    viridis = mpl.colormaps['viridis'].resampled(256)
    newcolors = viridis(np.linspace(0, 1, 256))
    pink = np.array([248/256, 24/256, 148/256, 1])
    black = np.array([0/256, 0/256, 0/256, 1])
    newcolors[-16:, :] = pink
    newcolors[:16,:] = black


    ax = seaborn.heatmap(vstup,annot =False, cmap =  mpl.colors.ListedColormap(newcolors))
    ax.set(xlabel="", ylabel="",title=title)
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
    if(exportName != ""):
        plt.savefig(exportName,bbox_inches="tight")
        plt.close()
    else:
        plt.show()
    #TODO: SaveFig
    #plt.savefig(".pdf")

In [5]:
import pickle
class Embeddings_ready():
    
    arr_embeddings = []
    strings = []
    
    def __init__(self, path:str = "",embeddings= [],text:str =""):
        if(path != ""):
            self.loadPickle(path)
        elif(text != ""):
            self.arr_embeddings = self.prepEmbeddings(text)

        elif(len(embeddings)>0):
            self.arr_embeddings = embeddings
            
        
    def loadPickle(self,path:str):
        with open(path,"rb") as p:
            self.arr_embeddings= pickle.load(p)
        
    def savePickle(self, path:str):
        with open(path,"wb") as p:
            return pickle.dump(self.arr_embeddings,p)
        
    def getEmbeddings(self):
        return self.arr_embeddings
    
    #firstSEC, secondSEC - označuje zda používáme z embeddings "pos1" nebo "pos2" sloupec - 0 = pos1, 1 = pos2
    def getMapReadyEmbeddings(self, anotherClass = None,firstSEC:int = 1, secondSEC:int = 0):
        retVal = []
        if(anotherClass == None):
            anotherClass = self
        
       
        for i in range(len(self.arr_embeddings[0])):
            retVal.append([])
            #print(len(retVal))
            for j in range(len(anotherClass.arr_embeddings[0])):
                retVal[i].append([self.arr_embeddings[firstSEC][i:i+1],anotherClass.arr_embeddings[secondSEC][j:j+1]])
                #retVal[i].
                next
        return retVal
                
    def prepEmbeddings(self,text:str,n_Chars:int = 1):
        
        splitted = textSplitter(text,str(n_Chars))
        self.strings = splitted

        embedded = vlastni_vectorize_batch(splitted)
        return embedded
    
    
    def generateMap(self,matrix=None,firstSEC:int = 1, secondSEC:int = 0):
        if(matrix == None):
            matrix = self.getMapReadyEmbeddings(firstSEC=firstSEC, secondSEC=secondSEC)
        retVal = []
        for i in range(np.size(matrix,0)):
            retVal.append([])
            for j in range(np.size(matrix,1)):
                retVal[i].append(modelDist(matrix[i][j])[1][0][0].numpy()[0])
        #print(modelDist(matrix[i][j])[1][0][0].numpy()[0])
        return retVal
        
        
        #Asi zatím nemá smysl, aby existoval:
    def getWords(self,number:int = 0):
        if(number == 0):
            if(self.words1==None):
                return None #Asi by měl Rasinout exception?
            return self.words1


        elif(number == 1):
            if(self.words2 == None):
                if(self.words1 == None):
                    return None #Asi by měl Rasinout exception?
                return self.words1
            return self.words2
    

In [ ]:
import os
directory = "./Pickles/Words/"
overwrite = False


for i in os.listdir(directory):
    if(not i.endswith(".pickle")):
        print(i)
        continue
    f = os.path.join(directory,i)
    print(f)
    if(os.path.isfile(f)):
        try:
            if((not os.path.isfile(f.replace(".pickle",".png"))) or overwrite):
                s = pickle.load(open(f,'rb'))
                slova = textSplitter(s.words.replace('\n',''),str(1))
                Mapa(pd.DataFrame(data = s.vzdalenostniMapa,
                            index =  slova,
                            columns =  slova
                            ),f.replace(".pickle",".png"),s.words)
                            #f.replace(".pickle",".png")
             
        except Exception as e:
            print("wtf2 : " + str(e) )
            continue
    else:
        print("wtf")

        

In [ ]:
# s = pickle.load(open("./Pickles/old/868843c76aea94b09d28fa6d64b0292e8b638c20f90e90deee7b382619679aab1d6a19ece5cac33f5460196c15a7a5a48c60c189520b24b8d21d9faadec4ef3a.pickle","rb"))

Následující funkce počítá průměrné hodnoty jednotlivých diagonál dle relativní odchylky od hlavní diagonály.

In [6]:


import os
from statistics import mean
directory = "./Pickles/ProteinCLS/"
overwrite = False

maxDelka = 0 
slovnikOffsetuMatice = []
fileCounter = 0
for i in os.listdir(directory):
    if(not i.endswith(".pickle")):
        #print(i)
        continue
    f = os.path.join(directory,i)
    #print(f)

    if(os.path.isfile(f)):
        try:
            s = pickle.load(open(f,'rb'))
            vstupniText = s.words
            #print(vstupniText)
            

            
            #print(s.vzdalenostniMapa())
            matice = np.array(s.getVzdalenostniMapa())
            delka = np.size(matice, 0)
            maxDelka = max(maxDelka,delka)
            for offst in range(-delka+1, delka-1):
                slovnikOffsetuMatice.append([offst,matice.diagonal(offst).sum()/delka , fileCounter])
                #print(matice.diagonal(offst).sum() / delka)
            #print(slovnikOffsetuMatice)
            fileCounter+=1

            
            #print(matice)
            #break

            #num_rows, num_cols = matice.shape()

            #print(matice.diagonal(num_rows))
            #print("..")
            

             
        except Exception as e:
            print("wtf2 : " + str(e) )
            continue
    else:
        print("wtf")


sortedArray = sorted(slovnikOffsetuMatice, key=lambda x: x[1],reverse= True)

#print(sortedArray[0])
minimalVal  = np.min(sortedArray,).astype(int)
maxVal = np.max(sortedArray,).astype(int)

arr_sum = np.zeros(maxVal-minimalVal+2,).tolist()
arr_count = np.zeros(maxVal-minimalVal+2,).tolist()


for i in sortedArray:
    #print(i[1])
    arr_sum[-minimalVal+i[0]] += i[1]
    arr_count[-minimalVal+i[0]] += 1


#print(arr_sum)
prumery = []

for i in range(0,len(arr_sum)):
    #print(i)
    if(arr_count[i] == 0):
        #print(arr_sum[i],arr_count[i])
        continue
    prumery.append([i+minimalVal , arr_sum[i] /  arr_count[i]])

prumerySerazeno =sorted(prumery, key=lambda x: x[1],reverse= True)


print()



In [ ]:
#Export výsledků do formátu jednoduše zobrazitelného matlabem
kekw = np.array(sorted(prumery, key=lambda x: x[0]))
l = np.transpose(kekw).tolist()
print("x = " ,l[0],";")
print("y = " ,l[1],";")

TOHLE JE ŠÍLENSTVÍ - EXTRÉMNĚ MNOHO DAT!!!

In [25]:


import os
from statistics import mean
from collections import Counter

directory = "./Pickles/Words/"
overwrite = False

serazeniPpsti = None

maxDelka = 0 
slovnikOffsetuMatice = []

#counts [i] = counts.get(i, 0) + 1
slovnikPoctu = dict()
slovnikPpsti = dict()

fileCounter = 0
for i in os.listdir(directory):
    if(not i.endswith(".pickle")):
        #print(i)
        continue
    f = os.path.join(directory,i)
    #print(f)

    if(os.path.isfile(f)):
        try:
            s = pickle.load(open(f,'rb'))
            vstupniText = s.words
            #print(vstupniText)
            

            
            #print(s.vzdalenostniMapa())
            matice = np.array(s.getVzdalenostniMapa())
            delka = np.size(matice, 0)            

            #Projdi všechny výsledky matice
            
            for g in range(0,delka): #Vertical
                for h in range(0,delka): #Horizontal
                    #Každou hodnotu přičti do slovníků všech možných proměnných
                    for j in range(1,6):
                        subString = vstupniText[g+30-j:g+30]+vstupniText[h:h+j]
                        #print(subString)
                        #print(vstupniText)
                        slovnikPoctu[subString] = slovnikPoctu.get(subString,0)+1
                        slovnikPpsti[subString] = slovnikPoctu.get(subString,0)+matice[g,h]

                        #print(slovnikPoctu)
                        #print(slovnikPpsti)
                        #return
            print(".",end="")           






            #print(matice.diagonal(offst).sum() / delka)
            #print(slovnikOffsetuMatice)
            fileCounter+=1

            
            #print(matice)
            #break

            #num_rows, num_cols = matice.shape()

            #print(matice.diagonal(num_rows))
            #print("..")
            

            
        except Exception as e:
            print("wtf2 : " + str(e) )
            continue
    else:
        print("wtf")
    




...........................................................................................................

In [28]:
print(len(slovnikPoctu))
slovnikPrumerPpsti = dict()
pocetSpojeni = sum(slovnikPoctu.values())

for i in slovnikPpsti.keys():
    slovnikPrumerPpsti[i] = slovnikPpsti[i] / pocetSpojeni


74918865


In [ ]:
slovnikPpsti = None
slovnikPoctu = None
serazeniPpsti = sorted(slovnikPrumerPpsti.items(), key = lambda x:x[1])

In [32]:
serazeniPpsti = sorted(slovnikPrumerPpsti.items(), key = lambda x:x[1], reverse=True)

In [60]:
print(serazeniPpsti[3
                    
                    ])

(' o', 0.0019429110337580004)


In [49]:
soucet = 0
soucet2 = 0
pocet = 0
pocet2 = 0
for i in serazeniPpsti:
    if(" " in i[0][int(len(i[0])/2) : int(len(i[0])/2)+1 ]):
        soucet += i[1]
        pocet+=1
    else:
        soucet2 += i[1]
        pocet2+= 1


In [50]:
print(soucet)
print(soucet2)
print(pocet)
print(pocet2)

0.1745078081468546
1.017596839827804
9364668
65554197


In [51]:
print("Souvislost na hranici slov =" + str(soucet/pocet))
print("Souvislost uprostřed slov =" + str(soucet2/pocet2))

Souvislost na hranici slov =1.863470313596324e-08
Souvislost uprostřed slov =1.5522985352529052e-08


In [ ]:
counts = dict()
for i in items:
  counts[i] = counts.get(i, 0) + 1

In [ ]:
print(len(np.zeros((3,)).tolist()))


In [ ]:
kekw2 = np.zeros((2,2))
print(kekw2.tolist())

In [ ]:
print(len(arr_sum))

In [ ]:
# #print(s.words)##
# #print(textSplitter(s.words.replace('\n',' '),str(1)))
# print(s.toString())
# slova = textSplitter(s.words,str(1))

# Mapa(pd.DataFrame(data = s.vzdalenostniMapa,
#                            index =  slova,
#                            columns =  slova
#                           ),"./Pickles/old/868843c76aea94b09d28fa6d64b0292e8b638c20f90e90deee7b382619679aab1d6a19ece5cac33f5460196c15a7a5a48c60c189520b24b8d21d9faadec4ef3a.png")

In [ ]:
import numpy as np
vysledky = []
#with open("./CC-aug2018-oct2021_ces.10G_val.tsv","r",encoding = "utf8") as reader:
   #uniref90_tax-free_shuffled_val.tsv

soubory = {"./uniref90_tax-free_shuffled_val.tsv","./uniref90_tax-free_shuffled_train.tsv" }

soubory = {"./CC-aug2018-oct2021_ces.10G_train.tsv","./CC-aug2018-oct2021_ces.10G_val.tsv"}
meziVysledky = []
megas = 0
counter = 0 
for x in soubory:
   with open(x,"r",encoding = "utf8") as reader:
      for i in reader:
         length = len(i.split("\t")[-1])
         if(length == 1):
            continue
         meziVysledky.append(length)
         counter += 1
         if(counter % 1_000_000 == 0):
            print(".",end="")
            numpyVysledky = np.array(meziVysledky)
            megas+=1
            vysledky.append([megas,np.median(numpyVysledky),np.max(numpyVysledky), np.min(numpyVysledky), np.average(numpyVysledky), np.sum(numpyVysledky)])
            meziVysledky = []
      reader.close()
   print("\n-----\n")

numpyVysledky = np.array(meziVysledky)
megas+=(counter%1_000_000)/1_000_000
vysledky.append([megas,np.median(numpyVysledky),np.max(numpyVysledky), np.min(numpyVysledky), np.average(numpyVysledky), np.sum(numpyVysledky)])

In [ ]:
numpyVysledky = np.array(vysledky)
print("celkem:", counter)
print(np.median(numpyVysledky),np.max(numpyVysledky), np.min(numpyVysledky), np.average(numpyVysledky), np.sum(numpyVysledky))

vysledky[-1][0] -= 115

In [ ]:
print(counter)

In [ ]:
vysledky[-1][0] -= 1

In [ ]:
#for i in vysledky:
#   print(i)

print(np.max(vysledky[:][0]))

In [ ]:
vysl = np.array(vysledky)
#print(vysl[:,0])

print("median = ",np.median(vysl[:,1]), "max = ",np.max(vysl[:,2]),"; min = ", np.min(vysl[:,3]) , "; avg = ",  np.average(vysl[:,4]) ,"; sum = ", np.sum(vysl[:,5])   )

In [ ]:
for i in vysl:
    if(i[2 ] == 45355 ):
        print(i)

In [ ]:
meziVysledky = []
megas = 0
counter = 0 
for x in soubory:
   with open(x,"r",encoding = "utf8") as reader:
      for i in reader:
         if(len(i.split("\t")[-1])== 1):
           print("wtf?!" + i)
           break
      reader.close()
   print("\n-----\n")

In [ ]:
print(vysl[:,1])

In [ ]:

soubory = {"./CC-aug2018-oct2021_ces.10G_train.tsv","./CC-aug2018-oct2021_ces.10G_val.tsv"}
import re

pattern = re.compile(r"[äöüßÄÖÜẞ]+",re.IGNORECASE)

pismenka = {'ü','ö','ä','ß',''}
for x in soubory:
   with open(x,"r",encoding = "utf8") as reader:
      for i in reader:
         if(pattern.search(i)):
           print("wtf?!" + i)
           
      reader.close()
   print("\n-----\n")

In [ ]:
import os
directory = "./Pickles/Words/"
overwrite = False


for i in os.listdir(directory):
    if(not i.endswith(".pickle")):
        print(i)
        continue
    f = os.path.join(directory,i)
    print(f)
    if(os.path.isfile(f)):
        try:
            if((not os.path.isfile(f.replace(".pickle",".png"))) or overwrite):
                s = pickle.load(open(f,'rb'))
                slova = textSplitter(s.words.replace('\n',''),str(1))
                Mapa(pd.DataFrame(data = s.vzdalenostniMapa,
                            index =  slova,
                            columns =  slova
                            ),f.replace(".pickle",".png"),s.words)
                            #f.replace(".pickle",".png")
             
        except Exception as e:
            print("wtf2 : " + str(e) )
            continue
    else:
        print("wtf")
